In [1]:
import pickle
import os
import xml.etree.ElementTree as ET
import pandas as pd
import re
import shutil

In [2]:
def match_titles(title1, title2):
    title1 = re.sub(r'[\W_]+', '', title1).lower()
    title2 = re.sub(r'[\W_]+', '', title2).lower()
    if title1 in title2 or title2 in title1:
        return True
    return False

In [3]:
def get_baselines():
    f = open("./annotation/annotations2")
    data = []
    for line in f:
        data.append(line)

    dataset = {}
    for line in data:
        x = line.split("_")
        dataset[x[0]] = x[1:]
        
    f = open("./annotation/django_annotations")
    data = []
    for line in f:
        data.append(line)
        
    for line in data :
        x = line.split("_")
        if(len(x)>1) :
            dataset[x[0]] = x[1:]
            
    df = pd.read_excel("./annotation/Baselines.xlsx")
    ar = []
    for i in range(586) :
        if(df['Baseline Papers'][i]!='None' and type(df['Baseline Papers'][i])!=type(0.1)):
            ar.append(i)
    
    for ind in ar :
        paperId = df['PaperId'][ind]
        if(len(paperId)!=8):
            paperId = paperId[:-1]      
        x = df['Baseline Papers'][ind].split("_")
        
        dataset[paperId] = x
            
    return dataset

In [4]:
baselines = get_baselines()

In [5]:
len(baselines.keys())

1445

In [6]:
def get_citations_tags(baselines) :
    citation_list = {}
    for file in os.listdir("xmls_total/") :
        tree = ET.parse("xmls_total/"+file)
        root = tree.getroot()
        id = file[:8]
        if id in baselines :
            for element in root.iterfind("algorithm"):
                if(element.attrib['name']=="ParsCit"):
                    citlist = element.getchildren()
                    cits = citlist[0].getchildren()
                    citations = []
                    for cit in cits:
                        cit_dict = {}
                        if(cit.attrib['valid']=="true"):
                            try :
                                title = cit.find('title').text.lower()
                            except :
                                title = cit.find('rawString').text.lower()
                            cit_dict['paper_name'] = title
                            check = 0
                            for base in baselines[id] :
                                if(match_titles(base, title)):
                                    check = 1
                                    cit_dict['tag'] = 1
                                    break
                            if(check==0) :
                                cit_dict['tag']= 0
                            concat_context = ""
                            for context in cit.findall("contexts/context"):
                                concat_context+=context.text.lower()+" "
                            cit_dict['context'] = concat_context
                            if(len(concat_context)!=0):
                                citations.append(cit_dict)
                    citation_list[id] = citations
                
    return citation_list

In [7]:
citations = get_citations_tags(baselines)

In [8]:
print(len(citations.keys()))

1389


In [9]:
non_baselines = []
baselines = []
dataset = []
map_nb = {}
map_b = {}
map_nb_opp = {}
map_b_opp = {}
for key in citations :
#     if(key in citations) :
#         print("a")
        papers = citations[key]
        for paper in papers :
            ar = []
            ar.append(paper['paper_name'])
            ar.append(paper['tag'])
            ar.append(paper['context'])
            dataset.append(ar)
            if(paper['tag']==0):
                non_baselines.append(paper['context'])
                map_nb[paper['paper_name']+"_"+key] = len(non_baselines)-1
                map_nb_opp[len(non_baselines)-1] = paper['paper_name']+"_"+key
            else :
                baselines.append(paper['context'])
                map_b[paper['paper_name']+"_"+key] = len(baselines)-1  
                map_b_opp[len(baselines)-1] = paper['paper_name']+"_"+key

In [10]:
print(len(non_baselines))

29201


In [11]:
train_mapping = pickle.load(open("pickles_data/filenames_mapping.pkl", "rb"))
test_mapping = pickle.load(open("pickles_data/filename_map_test.pkl", "rb"))

In [12]:
armap = []
for x in train_mapping :
    filename = x.split("/")[-1]
    class_name = x.split("/")[-2]
    if(class_name=="class0"):
        armap.append(map_nb_opp[int(filename.split(".")[0])])
    else :
        armap.append(map_b_opp[int(filename.split(".")[0])])        

In [13]:
armap_test = []
for x in test_mapping :
    filename = x.split("/")[-1]
    class_name = x.split("/")[-2]
    if(class_name=="class0"):
        armap_test.append(map_nb_opp[int(filename.split(".")[0])])
    else :
        armap_test.append(map_b_opp[int(filename.split(".")[0])])        

In [15]:
output_tr = pickle.load(open("pickles/bert_embedar_tr_512.pkl", "rb"))
output_ts = pickle.load(open("pickles/bert_embedar_ts_512.pkl", "rb"))

In [16]:
dataset_bert = {}
for i in range(len(output_tr)):
    id = armap[i].split("_")[1]
    title = armap[i].split("_")[0]
    if(id not in dataset_bert):
        dataset_bert[id] = []
    data = {}
    data['paper_name'] = title
    data['embedding'] = output_tr[i]
    dataset_bert[id].append(data)
    
for i in range(len(output_ts)):
    id = armap_test[i].split("_")[1]
    title = armap_test[i].split("_")[0]
    if(id not in dataset_bert):
        dataset_bert[id] = []
    data = {}
    data['paper_name'] = title
    data['embedding'] = output_ts[i]
    dataset_bert[id].append(data)  

In [17]:
len(dataset_bert.keys())

1367

In [18]:
pickle.dump(dataset_bert, open("pickles_data/bert_embeddings_512.pkl", "wb"))